# Minimal dummy example with ictdeploy
## Creating a co-simulation graph
We consider the following two nodes graph as a minimal dummy example of two nodes exchanging a variable and randomly add or substract a given value from it:

<img src="./figures/model.png" alt="Model" style="width: 500px;"/>

The above graph can be created using ictdeploy as follow:

In [ ]:
import os
from ictdeploy import Simulator as Sim

In [ ]:
#create empty simulator
sim = Sim()

In [ ]:
#create Meta-model
sim.edit.add_meta(
        name="BaseMeta",
        set_attrs=["sou.T_in"],
        get_attrs=["senTemOut.T"]
    )

In [ ]:
#create Meta-model
sim.edit.add_meta(
        name="Base2Meta",
        set_attrs=["Tsupply"],
        get_attrs=["Tindoor"]
    )

In [ ]:
#create Model
sim.edit.add_model(
        name="BaseModel",
        meta="BaseMeta",
        image="",
        wrapper=os.path.join("wrappers", "base_wrap.py"),
        command=None,
        files=[os.path.join("files_to_add", "IBPSA_Fluid_FixedResistances_Examples_PlugFlowPipe.fmu")]
    )

In [ ]:
#create Model
sim.edit.add_model(
        name="Base2Model",
        meta="Base2Meta",
        image="integrcity/ict-simple",
        wrapper=os.path.join("wrappers", "building_wrap.py"),
        command=None,
        files=[os.path.join("files_to_add", "empty_file.txt")]
    )

Now that our `Meta-models` and `Models` have been created, we can populate our `Simulator` with `Nodes` as follow:

In [ ]:
#add node
sim.edit.add_node(
        name="Base0",
        model="BaseModel",
        init_values={},
        is_first=True,
        is_local=True
    )

In [ ]:
#add node
sim.edit.add_node(
        name="Base1",
        model="Base2Model",
        init_values={}        
    )

And add `Links` between the created `Nodes` as follow:

In [ ]:
sim.edit.add_link(get_node="Base0", get_attr="senTemOut.T", set_node="Base1", set_attr="Tsupply")
sim.edit.add_link(get_node="Base1", get_attr="Tindoor", set_node="Base0", set_attr="sou.T_in")

## Data structure
The created co-simulation graph can be described by the following tables, gathering information about `Nodes` and `Links`.

In [ ]:
sim.edit.nodes

In [ ]:
sim.edit.links

## Simulation parameters
Based on the created co-simulation graph, we can now create `Groups`, `Sequence` and `Steps` for the simulation as follow:

In [ ]:
grp0 = sim.create_group("Base0")
grp1 = sim.create_group("Base1")

In [ ]:
sim.create_sequence(grp0, grp1)
sim.create_steps([60] * 60)

## Running simulation
Everything is ready, we just have to run the simulation using `Simulator.run_simulation()`

In [ ]:
logs = sim.run_simulation(server="server.py")

In [ ]:
print(logs["nodes"]["Base0"].replace("172.17.01", "localhost"))

In [ ]:
for l in sim.get_logs(logs["orc"]):
    if "INFO" in l:
        print(l)

## Retrieving results
Now that the simulation is finished, we can connect to the database of results and list all the available stored values as follow:

In [ ]:
sim.results.connect_to_results_db(host="localhost")
sim.results.list_of_available_results

Based on the preceding available stored values, we can choose wich results to get using a pattern matching the name of the selected values.

In [ ]:
results = sim.results.get_results_by_pattern("OUT*")
results.keys()

And finnaly, we plot the selected results using `matplotlib`.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(9, 6))
plt.plot(results['OUT||Base0||senTemOut.T'], "-o", label="Base0 - senTemOut.T", alpha=0.5)
plt.plot(results['OUT||Base1||Tindoor'], "-o", label="Base1 - Tindoor", alpha=0.5)
plt.legend()